# Fazendo as inferências nas imagens

In [1]:
# Tentar rodar em paralelo - OK
# Abrir a imagem - OK
# gerando GrayScale - OK
# aplicar Scaler - OK
# gerar Filtro de Sobel - OK
# aplicar Scaler ao Sobel - OK
# gerar LBP - OK
# aplicar Scaler no LBP - OK
# separar as quadriculas - OK
# aplicar PCA
# Inferir se é faixa de pedestre    
# salvar imagem 1bit por SCM

## DEpois gerando dados

# agrupar em uma única imagem para toda a ciade
# agrupar dbScan
# gerar polígonos
# gerar linhas

In [2]:
import multiprocessing as mp
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np
import glob
import joblib
from skimage import filters
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import geopandas as gpd
from shapely.geometry import box
from sklearn.svm import SVC
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
gdf_scms = gpd.read_file('downloads/SIRGAS_SHP_quadriculaortofoto2017.zip!SIRGAS_SHP_quadriculaortofoto2017/SIRGAS_SHP_quadriculaortofoto2017.shp')

In [4]:
n_cpu = mp.cpu_count() - 1

In [5]:
sample =  gdf_scms[2:17]['qo2010_cod'].to_list()

In [6]:
len(sample)

15

In [7]:
sample

['3335-313',
 '2346-434',
 '3336-313',
 '3335-434',
 '3335-433',
 '3335-424',
 '3335-423',
 '3335-414',
 '3335-413',
 '3335-334',
 '3335-333',
 '3335-324',
 '3335-323',
 '3335-314',
 '2346-423']

In [8]:
radius = 3
n_points = 8 * radius
tamanho_janela = 24 # tamanho em pixels
sobreposicao_de_janelas = 12 # sobreposição das janelas em pixels

In [9]:
gdf_quadras = gpd.read_file('zip://downloads/SIRGAS_SHP_quadraMDSF.zip!/SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')

In [10]:
grayscale_scaler = joblib.load('pre-process/grayscale.scaler.save')
sobel_scaler = joblib.load('pre-process/sobel.scaler.save') 
lbp_scaler = joblib.load('pre-process/lbp.scaler.save')

In [11]:
pca = joblib.load('resultados/modelos-treinados/PCA-faixas-pedestre.pkl')

In [12]:
clf = joblib.load('resultados/modelos-treinados/SVC-faixas-pedestre.pkl')

In [17]:
def faixas(scm):
    file = glob.glob(f'../ortofotos-2017/RGB-2017/*{scm}*.jp2')
    dataset = rasterio.open(file[0])
    data, location = [], []

    grayscale = 0.2125 * dataset.read(1) + \
            0.7154 * dataset.read(2) + \
            0.0721 * dataset.read(3)
    grayscale = grayscale.astype('uint8')

    grayscale = grayscale_scaler.transform(grayscale.reshape(-1, 1)).reshape(grayscale.shape)

    sobel = filters.sobel(grayscale)
    sobel = sobel_scaler.transform(sobel.reshape(-1, 1)).reshape(sobel.shape)

    lbp = local_binary_pattern(grayscale, n_points, radius)
    lbp = lbp_scaler.transform(lbp.reshape(-1, 1)).reshape(lbp.shape)

    quadras_scm = gpd.clip(gdf_quadras, gpd.GeoSeries(box(*dataset.bounds))).unary_union

    quadras = features.rasterize(
        ((g, 255) for g in quadras_scm),
        out_shape=dataset.shape,
        transform=dataset.transform)

    for i in np.arange(0, grayscale.shape[0] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
        for j in np.arange(0, grayscale.shape[1] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
            # Se mais de 98% dos pixels estiverem fora da quadra
            if np.count_nonzero(quadras[i:i+tamanho_janela, j:j+tamanho_janela] == 255) < 0.02 * (tamanho_janela ** 2):
                # print(i, j)
                img_gray = grayscale[i:i+tamanho_janela, j:j+tamanho_janela]
                img_sobel = sobel[i:i+tamanho_janela, j:j+tamanho_janela]
                img_lbp = lbp[i:i+tamanho_janela, j:j+tamanho_janela]

                # Data Augmentation
                
                img = np.concatenate([
                    img_gray,
                    img_sobel,
                    img_lbp
                ]).reshape(1, 3 * (tamanho_janela ** 2))
        
                data.append(img.reshape(1, 3 * (tamanho_janela ** 2)))
                location.append([i, j])
            

    np_data = np.array(data).squeeze()
    np_data = pca.transform(np_data)

    ## TODO
    # fazer a predição
    y = clf.predict(np_data)

    matrix = np.zeros([dataset.height, dataset.width], dtype='uint8')

    for i, l in enumerate(location):
        ## TODO
        # persistir a predição
        matrix[l[0]:l[0] + tamanho_janela, l[1]:l[1] + tamanho_janela] = 1

    profile = dataset.profile
    profile['count'] = 1
    
    with rasterio.open(f'resultados/raster/faixas-{scm}.jp2', 'w', nbits=1, **profile) as dst:
        dst.write(matrix, indexes=1)

    # print(matrix[0:24, 0:24])
    return matrix

In [18]:
faixas('3335-313')

(109339, 1728) (109339, 1728)
(109339, 135)
(4895, 4519)
(4895, 4519)
{'driver': 'JP2OpenJPEG', 'dtype': 'uint8', 'nodata': None, 'width': 4519, 'height': 4895, 'count': 1, 'crs': CRS.from_epsg(31983), 'transform': Affine(0.11999999999994009, 0.0, 321621.30899,
       0.0, -0.11999999999994009, 7374093.18301), 'blockxsize': 1024, 'blockysize': 1024, 'tiled': True, 'interleave': 'pixel'}


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
np.count_nonzero(m)

In [ ]:
for i, l in enumerate([[0,1], [1,2], [2,3]]):
    print(m[l[0]:24, l[1]:24])

In [ ]:
m

In [ ]:
with mp.Pool(processes=n_cpu) as p:
    p.map(faixas, sample)